In [ ]:
# SP HEURÍSTICO

import pandas as pd
import time

costs = pd.read_excel(
    "/content/matrices_40.xlsx", sheet_name="Instancia_1", index_col=0
)


def greedy_tsp_dict(costs, start=None):
    start_time = time.time()
    nodes = list(costs.index)
    if start is None and nodes:
        start = nodes[0]
    elif start not in nodes:
        raise ValueError(f"Start node {start} not in costs index.")

    unvisited = set(nodes)
    path = [start]
    unvisited.remove(start)
    current = start
    total_cost = 0

    while unvisited:
        # Pick nearest unvisited node
        nxt = min(unvisited, key=lambda v: costs.loc[current, v])
        total_cost += costs.loc[current, nxt]
        current = nxt
        unvisited.remove(nxt)
        path.append(nxt)

    # Return to start
    total_cost += costs.loc[current, start]
    path.append(start)
    start_time -= time.time()

    return path, total_cost, -start_time


# use it like:
start_node = list(costs.keys())[0]
path, cost, time_ = greedy_tsp_dict(costs, start=start_node)
print("Greedy path:", path)
print("Greedy cost:", cost)
print("Greedy time:", time_)

# FOR MANY
costs_40 = pd.read_excel("/content/matrices_40.xlsx", sheet_name=None, index_col=0)
costs_100 = pd.read_excel("/content/matrices_100.xlsx", sheet_name=None, index_col=0)
costs_150 = pd.read_excel("/content/matrices_150.xlsx", sheet_name=None, index_col=0)
costs_200 = pd.read_excel("/content/matrices_200.xlsx", sheet_name=None, index_col=0)

casos = [costs_40, costs_100, costs_150, costs_200]
results = []
for index_, caso in enumerate(casos):
    temp_caso = []
    for nombre, df in caso.items():
        start_node = list(df.keys())[0]
        print(f"Starting case {index_} at instance {nombre}")
        path, cost, time_s = greedy_tsp_dict(df, start=start_node)
        temp_caso.append((path, cost, time_s))
    results.append(temp_caso)
    print(f"End case {index_}")


output_excel_file = "greedy_tsp_results.xlsx"
sheet_names = ["CASO_40", "CASO_100", "CASO_150", "CASO_200"]

with pd.ExcelWriter(output_excel_file) as writer:
    for i, caso_results in enumerate(results):
        # Create a DataFrame for each set of results
        df_sheet = pd.DataFrame(caso_results, columns=["PATH", "COST", "TIME (S)"])

        # Set the index names
        df_sheet.index = [f"INSTANCIA_{j+1}" for j in range(len(df_sheet))]

        # Write the DataFrame to a sheet
        df_sheet.to_excel(writer, sheet_name=sheet_names[i])

print(f"Results saved to {output_excel_file}")

import math
import time

# NEW GREEDY
def greedy_tsp_dict(costs, start):
    t0 = time.perf_counter()

    nodes = list(costs.keys())
    unvisited = set(nodes)
    path = [start]
    unvisited.remove(start)
    current = start
    total = 0

    while unvisited:
        nxt = min(unvisited, key=lambda v: costs[current][v])
        total += costs[current][nxt]
        current = nxt
        unvisited.remove(nxt)
        path.append(nxt)

    total += costs[current][start]
    path.append(start)

    t1 = time.perf_counter()
    return path, total, t1 - t0


def two_opt(path, costs, max_rounds=3):  # limit improvement rounds
    t0 = time.perf_counter()
    best_cost = tour_cost(path, costs)

    for _ in range(max_rounds):
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path) - 1):
                if j - i == 1:
                    continue
                new_path = path[:i] + path[i:j][::-1] + path[j:]
                new_cost = tour_cost(new_path, costs)
                if new_cost < best_cost:
                    path = new_path
                    best_cost = new_cost
                    improved = True
        if not improved:
            break
    t1 = time.perf_counter()
    return path, best_cost, t1 - t0


def tour_cost(path, costs):
    total = 0
    for i in range(len(path) - 1):
        total += costs[path[i]][path[i + 1]]
    return total


def improved_tsp(costs, sample_starts=5):
    t_total_start = time.perf_counter()

    nodes = list(costs.keys())
    best_path, best_cost = None, math.inf
    best_times = {}

    for start in nodes[:sample_starts]:
        p, c, greedy_time = greedy_tsp_dict(costs, start)
        p2, c2, opt_time = two_opt(p, costs)

        if c2 < best_cost:
            best_path, best_cost = p2, c2
            best_times = {"greedy_time_sec": greedy_time, "two_opt_time_sec": opt_time}

    t_total_end = time.perf_counter()
    best_times["total_time_sec"] = t_total_end - t_total_start

    return best_path, best_cost, best_times["total_time_sec"]


costs = pd.read_excel(
    "/content/matrices_40.xlsx", sheet_name="Instancia_1", index_col=0
)

path, cost, times = improved_tsp(costs, sample_starts=5)

print("Best Path:", path)
print("Best Cost:", cost)
print("Timing:", times)

costs_40 = pd.read_excel("/content/matrices_40.xlsx", sheet_name=None, index_col=0)
costs_100 = pd.read_excel("/content/matrices_100.xlsx", sheet_name=None, index_col=0)
costs_150 = pd.read_excel("/content/matrices_150.xlsx", sheet_name=None, index_col=0)
costs_200 = pd.read_excel("/content/matrices_200.xlsx", sheet_name=None, index_col=0)

casos = [costs_40, costs_100, costs_150, costs_200]
results = []
for index_, caso in enumerate(casos):
    temp_caso = []
    for nombre, df in caso.items():
        start_node = list(df.keys())[0]
        print(f"Starting case {index_} at instance {nombre}")
        path, cost, time_s = improved_tsp(df, sample_starts=5)
        temp_caso.append((path, cost, time_s))
    results.append(temp_caso)
    print(f"End case {index_}")


output_excel_file_2 = "greedy_tsp_improved_results.xlsx"
sheet_names = ["CASO_40", "CASO_100", "CASO_150", "CASO_200"]

with pd.ExcelWriter(output_excel_file_2) as writer:
    for i, caso_results in enumerate(results):
        # Create a DataFrame for each set of results
        df_sheet = pd.DataFrame(caso_results, columns=["PATH", "COST", "TIME (S)"])

        # Set the index names
        df_sheet.index = [f"INSTANCIA_{j+1}" for j in range(len(df_sheet))]

        # Write the DataFrame to a sheet
        df_sheet.to_excel(writer, sheet_name=sheet_names[i])

print(f"Results saved to {output_excel_file_2}")